## Context Manager w Pythonie

Context manager to mechanizm w Pythonie, który zapewnia automatyczne zarządzanie zasobami.

**Podstawowa składnia:**
```python
with expression as variable:
    # kod używający zasobu
# zasób automatycznie zwolniony
```

### Problem bez context managera

In [ ]:
# Ryzykowny kod - łatwo zapomnieć o close()
file = open('test.txt', 'w')
file.write('Hello World')
# Jeśli tutaj wystąpi błąd, plik nie zostanie zamknięty!
file.close()  # Możemy zapomnieć

**Problemy:**
- Łatwo zapomnieć o `close()`
- Jeśli wystąpi wyjątek, `close()` nie zostanie wywołane
- Wyciek zasobów (file descriptors, memory, connections)

### Rozwiązanie: context manager

In [ ]:
# Bezpieczny kod - automatyczne zamknięcie
with open('test.txt', 'w') as file:
    file.write('Hello World')
    # Nawet jeśli tutaj wystąpi błąd, plik zostanie zamknięty!

# Plik automatycznie zamknięty po wyjściu z bloku 'with'
print("Plik zamknięty automatycznie")

**Zalety:**
- Automatyczne zamykanie zasobu
- Gwarancja zwolnienia zasobu nawet przy wyjątku
- Czytelniejszy kod

### Jak działa context manager, czyli protokół context managera

Context manager to obiekt z dwiema metodami:
- `__enter__()` - wywoływana na początku bloku `with`
- `__exit__()` - wywoływana na końcu bloku `with` (nawet przy wyjątku)

In [1]:
class FileManager:
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        print(f"Otwieranie pliku: {self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file
    
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Zamykanie pliku: {self.filename}")
        if self.file:
            self.file.close()
        # Return False by default - propaguj wyjątki
        return False

# Użycie
with FileManager('example.txt', 'w') as f:
    f.write('Test content')
    print("Praca z plikiem")

print("Koniec - plik zamknięty")

Otwieranie pliku: example.txt
Praca z plikiem
Zamykanie pliku: example.txt
Koniec - plik zamknięty


### Context manager z wyjątkiem

In [2]:
# Nawet gdy wystąpi wyjątek, __exit__ zostanie wywołane
with FileManager('error_test.txt', 'w') as f:
    f.write('Start')
    raise ValueError("Symulowany błąd")
    f.write('End')  # Ta linia się nie wykona

print("Mimo błędu, plik został zamknięty")

Otwieranie pliku: error_test.txt
Zamykanie pliku: error_test.txt


ValueError: Symulowany błąd

### Prostsze tworzenie: @contextmanager decorator

In [3]:
from contextlib import contextmanager

@contextmanager
def file_manager(filename, mode):
    print(f"Otwieranie: {filename}")
    file = open(filename, mode)
    try:
        yield file  # To trafia do 'as variable'
    finally:
        print(f"Zamykanie: {filename}")
        file.close()

# Użycie identyczne jak klasa
with file_manager('decorator_test.txt', 'w') as f:
    f.write('Test z dekoratorem')

Otwieranie: decorator_test.txt
Zamykanie: decorator_test.txt


**Anatomia @contextmanager:**
- Kod przed `yield` = `__enter__()`
- `yield value` = zwraca wartość do `as variable`
- Kod po `yield` (w `finally`) = `__exit__()`

### Praktyczne przykłady użycia

#### 1. Pomiar czasu wykonania

In [4]:
import time
from contextlib import contextmanager

@contextmanager
def timer(name):
    start = time.time()
    print(f"Start: {name}")
    yield
    end = time.time()
    print(f"Koniec: {name}, czas: {end - start:.4f}s")

# Użycie
with timer("Operacja 1"):
    time.sleep(0.1)
    sum(range(1000000))

Start: Operacja 1
Koniec: Operacja 1, czas: 0.1351s


#### 2. Tymczasowa zmiana katalogu

In [5]:
import os
from contextlib import contextmanager

@contextmanager
def change_directory(path):
    original = os.getcwd()
    print(f"Zmiana z {original} na {path}")
    os.chdir(path)
    try:
        yield
    finally:
        print(f"Powrót do {original}")
        os.chdir(original)

# Użycie
print(f"Aktualny katalog: {os.getcwd()}")

with change_directory('..'):
    print(f"Katalog wewnątrz with: {os.getcwd()}")

print(f"Katalog po wyjściu: {os.getcwd()}")

Aktualny katalog: C:\Users\User\PycharmProjects\design_patterns_3\desing-patterns-materials
Zmiana z C:\Users\User\PycharmProjects\design_patterns_3\desing-patterns-materials na ..
Katalog wewnątrz with: C:\Users\User\PycharmProjects\design_patterns_3
Powrót do C:\Users\User\PycharmProjects\design_patterns_3\desing-patterns-materials
Katalog po wyjściu: C:\Users\User\PycharmProjects\design_patterns_3\desing-patterns-materials


#### 3. Połączenie z bazą danych

In [6]:
from contextlib import contextmanager

class Database:
    def __init__(self, connection_string):
        self.connection_string = connection_string
        self.connected = False
    
    def connect(self):
        print(f"Łączenie z {self.connection_string}")
        self.connected = True
    
    def disconnect(self):
        print(f"Rozłączanie z {self.connection_string}")
        self.connected = False
    
    def query(self, sql):
        if not self.connected:
            raise Exception("Nie połączono z bazą")
        print(f"Wykonuję: {sql}")
        return ["wynik1", "wynik2"]

@contextmanager
def database_connection(connection_string):
    db = Database(connection_string)
    db.connect()
    try:
        yield db
    finally:
        db.disconnect()

# Użycie
with database_connection("postgresql://localhost/mydb") as db:
    results = db.query("SELECT * FROM users")
    print(f"Wyniki: {results}")

print("Połączenie automatycznie zamknięte")

Łączenie z postgresql://localhost/mydb
Wykonuję: SELECT * FROM users
Wyniki: ['wynik1', 'wynik2']
Rozłączanie z postgresql://localhost/mydb
Połączenie automatycznie zamknięte


#### 4. Transakcje bazodanowe

In [7]:
@contextmanager
def transaction(db):
    print("BEGIN TRANSACTION")
    try:
        yield db
        print("COMMIT")
    except Exception as e:
        print(f"ROLLBACK - błąd: {e}")
        raise

# Sukces
print("Test 1: Sukces")
db = Database("postgresql://localhost/mydb")
db.connect()
with transaction(db):
    db.query("INSERT INTO users VALUES (1, 'Jan')")
    db.query("INSERT INTO users VALUES (2, 'Anna')")

# Błąd
print("\nTest 2: Błąd")
try:
    with transaction(db):
        db.query("INSERT INTO users VALUES (3, 'Piotr')")
        raise ValueError("Coś poszło nie tak")
        db.query("INSERT INTO users VALUES (4, 'Maria')")  # Nie wykona się
except ValueError:
    print("Transakcja wycofana")

db.disconnect()

Test 1: Sukces
Łączenie z postgresql://localhost/mydb
BEGIN TRANSACTION
Wykonuję: INSERT INTO users VALUES (1, 'Jan')
Wykonuję: INSERT INTO users VALUES (2, 'Anna')
COMMIT

Test 2: Błąd
BEGIN TRANSACTION
Wykonuję: INSERT INTO users VALUES (3, 'Piotr')
ROLLBACK - błąd: Coś poszło nie tak
Transakcja wycofana
Rozłączanie z postgresql://localhost/mydb


### Parametry __exit__()

`__exit__(self, exc_type, exc_value, traceback)`:

- `exc_type` - typ wyjątku (None jeśli brak błędu)
- `exc_value` - wartość wyjątku
- `traceback` - traceback obiektu

**Zwracana wartość:**
- `False` lub `None` - propaguj wyjątek dalej (domyślnie)
- `True` - złap wyjątek (suppress)

In [9]:
class SuppressErrors:
    def __enter__(self):
        print("Wejście do bloku")
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"Wyjście: exc_type={exc_type}, exc_value={exc_value}")
        if exc_type is ValueError:
            print("Tłumimy ValueError")
            return True  # Suppress ValueError
        return False  # Propaguj inne wyjątki


In [10]:
# Test 1: ValueError zostanie złapany
print("Test 1: ValueError")
with SuppressErrors():
    print("Wewnątrz bloku")
    raise ValueError("Test error")
    print("Ta linia się nie wykona")
print("Program kontynuuje - ValueError został stłumiony\n")


Test 1: ValueError
Wejście do bloku
Wewnątrz bloku
Wyjście: exc_type=<class 'ValueError'>, exc_value=Test error
Tłumimy ValueError
Program kontynuuje - ValueError został stłumiony



In [11]:
# Test 2: TypeError zostanie rozpropagowany
print("Test 2: TypeError")
with SuppressErrors():
        raise TypeError("Inny błąd")


Test 2: TypeError
Wejście do bloku
Wyjście: exc_type=<class 'TypeError'>, exc_value=Inny błąd


TypeError: Inny błąd

### Wbudowane context managery

Python ma wiele wbudowanych context managerów:

**Pliki:**
```python
with open('file.txt') as f:
    content = f.read()
```

**Locki (threading):**
```python
import threading
lock = threading.Lock()
with lock:
    # critical section
```

**Tłumienie wyjątków:**
```python
from contextlib import suppress
with suppress(FileNotFoundError):
    os.remove('nonexistent.txt')
```

**Przekierowanie stdout:**
```python
from contextlib import redirect_stdout
with open('output.txt', 'w') as f:
    with redirect_stdout(f):
        print('To trafi do pliku')
```

### Kiedy używać context managera?

**Używaj context managera gdy:**
- Zarządzasz zasobami wymagającymi cleanup (pliki, połączenia, locki)
- Potrzebujesz setup + teardown (np. pomiar czasu)
- Chcesz zagwarantować wykonanie kodu cleanup nawet przy wyjątku

**Przykłady:**
- Pliki
- Połączenia z bazą danych
- Locki i semafory
- Transakcje
- Tymczasowe zmiany stanu (katalog, zmienne środowiskowe)
- Pomiar czasu/zasobów

### Podsumowanie

**Context manager:**
- Zapewnia automatyczne zarządzanie zasobami
- Składnia: `with expression as variable:`
- Implementacja: klasa z `__enter__` i `__exit__` lub funkcja z `@contextmanager`
- Gwarantuje cleanup nawet przy wyjątku

**Dwie metody tworzenia:**
1. Klasa z `__enter__` i `__exit__` (bardziej kontroli)
2. Funkcja z `@contextmanager` i `yield` (prostsze)

**Zalety:**
- Automatyczne zarządzanie zasobami
- Gwarancja cleanup
- Czytelny kod
- Bezpieczniejszy kod (brak wycieków zasobów)